In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import pandas_datareader as pdr
import yfinance as yf
from datetime import datetime



In [ ]:
from pandas_datareader import data as pdr
import yfinance as yf
yf.pdr_override()
y_symbols = ['SCHAND.NS', 'TATAPOWER.NS', 'ITC.NS']
startdate = datetime(2022,12,1)
enddate = datetime(2022,12,15)
data = pdr.get_data_yahoo(y_symbols, start=startdate, end=enddate)

In [ ]:
ticker=['AAPL']
yf.pdr_override()
st=datetime(2010,1,1)
en=datetime(2020,12,1)
df = pdr.get_data_yahoo('AAPL',start=st,end=en) 
df.head()

In [ ]:
dfwd=df.reset_index()
dfwd

In [ ]:
dfwd=dfwd.drop(['Date','Adj Close'],axis=1)
dfwd

In [ ]:
plt.plot(dfwd.Close)

In [ ]:
ma100=dfwd.Close.rolling(100).mean()
ma100
#Should be 100 or 200 days moving average

In [ ]:
plt.figure(figsize=(12,6))
plt.plot(dfwd.Close)
plt.plot(ma100,'r')

In [ ]:
dfwd.shape

In [ ]:
#Splitting data into training and testing

datatrain=pd.DataFrame(dfwd['Close'][0:int(len(dfwd)*0.70)])
datatesting=pd.DataFrame(dfwd['Close'][int(len(dfwd)*0.70):int(len(df))])
print(datatrain.shape)
print(datatesting.shape)

In [ ]:
from sklearn.preprocessing import MinMaxScaler
scaler=MinMaxScaler(feature_range=(0,1))

In [ ]:
newdatatrain=scaler.fit_transform(datatrain)
newdatatrain

In [ ]:
x_train=[]
y_train=[]

for i in range(100,datatrain.shape[0]):
    x_train.append(newdatatrain[i-100: i])
    y_train.append(newdatatrain[i,0])

x_train , y_train =np.array(x_train),np.array(y_train)
x_train.shape

In [ ]:
from keras.layers import Dense,Dropout,LSTM
from keras.models import Sequential

In [ ]:
model=Sequential()
model.add(LSTM(units=50,activation='relu',return_sequences=True,input_shape=(x_train.shape[1],1)))
model.add(Dropout(0.2))

model.add(LSTM(units=60,activation='relu',return_sequences=True))
model.add(Dropout(0.3))

model.add(LSTM(units=80,activation='relu',return_sequences=True))
model.add(Dropout(0.4))

model.add(LSTM(units=120,activation='relu'))
model.add(Dropout(0.5))

model.add(Dense(units=1))

In [ ]:
model.summary()

In [ ]:
model.compile(optimizer='adam',loss='mean_squared_error')
model.fit(x_train,y_train,epochs=50)

In [ ]:
model.save('datamod.h5')


In [ ]:
datatrain.tail(100)

In [ ]:
p100d=datatrain.tail(100)

p100d


In [ ]:
finaldata = pd.concat([p100d, datatesting], ignore_index=True)
finaldata


In [ ]:
inpd=scaler.fit_transform(finaldata)
inpd

In [ ]:
x_test=[]
y_test=[]

for i in range(100,inpd.shape[0]):
    x_test.append(inpd[i-100: i])
    y_test.append(inpd[i,0])

In [ ]:
x_test,y_test=np.array(x_test),np.array(y_test)

In [ ]:
#making predictions

In [ ]:
y_pred=model.predict(x_test)
y_pred.shape

In [ ]:
y_pred

In [ ]:
scaler.scale_


In [ ]:
scalefactor=1/0.02099517
y_pred=y_pred*scalefactor
y_test=y_test*scalefactor


In [ ]:
plt.figure(figsize=(12,6))
plt.plot(y_test,'b',label='Original Price')
plt.plot(y_pred,'r',label='predicted Price')
plt.show()